In [582]:
import pandas as pd
from sqlalchemy import create_engine

ENDPOINT = 'heicoders-playground.c2ced10ceyki.ap-southeast-1.rds.amazonaws.com'
PORT = 3306
USERNAME = 'student300'
PASSWORD = 'heicoders_AI300'
DBNAME = 'ai300_capstone'

database_conn = create_engine(f'mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DBNAME}')



In [583]:
query_churn_status = """
    SELECT * 
    FROM churn_status;
"""
churn_status_df = pd.read_sql(query_churn_status, database_conn)


query_account = """
    SELECT * 
    FROM account;
"""
account_df = pd.read_sql(query_account, database_conn)


query_account_usage = """
    SELECT * 
    FROM account_usage;
"""
account_usage_df = pd.read_sql(query_account_usage, database_conn)


query_customer = """
    SELECT * 
    FROM customer;
"""
customer_df = pd.read_sql(query_customer, database_conn)
display(customer_df.info())

query_city = """
    SELECT * 
    FROM city;
"""
city_df = pd.read_sql(query_city, database_conn)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     7043 non-null   object
 1   gender          7043 non-null   object
 2   age             7043 non-null   int64 
 3   senior_citizen  7043 non-null   object
 4   married         7043 non-null   object
 5   num_dependents  7043 non-null   int64 
 6   zip_code        7043 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 385.3+ KB


None

In [584]:
# Join tables

query_full = """
    SELECT  A.*,
            B.account_id,
            B.tenure_months,
            B.num_referrals,
            B.has_internet_service,
            B.internet_type,
            B.has_unlimited_data,
            B.has_phone_service,
            B.has_multiple_lines,
            B.has_premium_tech_support,
            B.has_online_security,
            B.has_online_backup,
            B.has_device_protection,
            B.contract_type,
            B.paperless_billing,
            B.payment_method,
            C.avg_long_distance_fee_monthly,
            C.total_long_distance_fee,
            C.avg_gb_download_monthly,
            C.stream_tv,
            C.stream_movie,
            C.stream_music,
            C.total_monthly_fee,
            C.total_charges_quarter,
            C.total_refunds,
            D.gender,
            D.age,
            D.senior_citizen,
            D.married,
            D.num_dependents,
            D.zip_code,
            E.city,
            E.population
    FROM churn_status AS A
    LEFT JOIN account AS B
    ON A.customer_id = B.customer_id
    LEFT JOIN account_usage AS C
    ON B.account_id = C.account_id
    LEFT JOIN customer AS D
    ON A.customer_id = D.customer_id
    LEFT JOIN city AS E
    ON D.zip_code = E.zip_code
"""

full_df = pd.read_sql(query_full, database_conn)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_id                    7043 non-null   object 
 1   status                         7043 non-null   object 
 2   churn_label                    7043 non-null   object 
 3   churn_category                 7043 non-null   object 
 4   churn_reason                   7043 non-null   object 
 5   account_id                     7043 non-null   object 
 6   tenure_months                  7043 non-null   int64  
 7   num_referrals                  7043 non-null   int64  
 8   has_internet_service           7043 non-null   object 
 9   internet_type                  7043 non-null   object 
 10  has_unlimited_data             7043 non-null   object 
 11  has_phone_service              7043 non-null   object 
 12  has_multiple_lines             7043 non-null   o

In [585]:
import plotly.express as px

fig = px.bar(full_df,x="churn_category")
fig.show()

In [586]:
fig = px.bar(full_df,x="churn_reason")
fig.show()

In [587]:
full_df["churn_category"].unique()

array(['Dissatisfaction', '', 'Competitor', 'Attitude', 'Price', 'Other'],
      dtype=object)

In [588]:
full_df["churn_reason"].unique()

array(['Network reliability', '', 'Competitor had better devices',
       'Attitude of support person',
       'Competitor offered higher download speeds',
       'Competitor made better offer', 'Long distance charges',
       'Deceased', 'Competitor offered more data',
       'Attitude of service provider', 'Price too high',
       'Product dissatisfaction', 'Service dissatisfaction',
       'Poor expertise of online support', "Don't know", 'Moved',
       'Extra data charges', 'Limited range of services',
       'Lack of affordable download/upload speed',
       'Poor expertise of phone support',
       'Lack of self-service on Website'], dtype=object)

In [589]:
# replace missing String values

full_df["churn_category"] = full_df["churn_category"].replace("", "Other")
full_df["churn_reason"] = full_df["churn_reason"].replace("", "Other")

display(full_df["churn_category"].unique())
display(full_df["churn_reason"].unique())

array(['Dissatisfaction', 'Other', 'Competitor', 'Attitude', 'Price'],
      dtype=object)

array(['Network reliability', 'Other', 'Competitor had better devices',
       'Attitude of support person',
       'Competitor offered higher download speeds',
       'Competitor made better offer', 'Long distance charges',
       'Deceased', 'Competitor offered more data',
       'Attitude of service provider', 'Price too high',
       'Product dissatisfaction', 'Service dissatisfaction',
       'Poor expertise of online support', "Don't know", 'Moved',
       'Extra data charges', 'Limited range of services',
       'Lack of affordable download/upload speed',
       'Poor expertise of phone support',
       'Lack of self-service on Website'], dtype=object)

In [590]:
categorical_cols = list(full_df.select_dtypes('object').columns)
print("Categorical columns:", categorical_cols)

for col in categorical_cols:
    print(col, full_df[col].unique())




Categorical columns: ['customer_id', 'status', 'churn_label', 'churn_category', 'churn_reason', 'account_id', 'has_internet_service', 'internet_type', 'has_unlimited_data', 'has_phone_service', 'has_multiple_lines', 'has_premium_tech_support', 'has_online_security', 'has_online_backup', 'has_device_protection', 'contract_type', 'paperless_billing', 'payment_method', 'stream_tv', 'stream_movie', 'stream_music', 'gender', 'senior_citizen', 'married', 'city']
customer_id ['0013-EXCHZ' '0014-BMAQU' '0016-QLJIS' ... '9938-TKDGL' '9938-ZREHM'
 '9964-WBQDJ']
status ['Churned' 'Stayed' 'Joined']
churn_label ['Yes' 'No' '']
churn_category ['Dissatisfaction' 'Other' 'Competitor' 'Attitude' 'Price']
churn_reason ['Network reliability' 'Other' 'Competitor had better devices'
 'Attitude of support person' 'Competitor offered higher download speeds'
 'Competitor made better offer' 'Long distance charges' 'Deceased'
 'Competitor offered more data' 'Attitude of service provider'
 'Price too high' 'Pro

In [591]:
# there are missing values in churn_label column
# drop missing values

full_df = full_df[full_df["churn_label"] != ""]


In [592]:
# Feature generation

# Has dependents or not
full_df.loc[full_df["num_dependents"] > 0, "has_dependents"] = 1
full_df["has_dependents"].fillna(0, inplace=True)
full_df["has_dependents"] = full_df["has_dependents"].astype(int)


In [593]:
# one hot encoding
column_pull = ["churn_label",
                  "contract_type", "has_dependents",
                  "total_refunds", "tenure_months",
                  "has_premium_tech_support", "num_referrals"]


onehot_columns = ["contract_type","has_dependents",
                  "has_premium_tech_support"]

use_df = full_df[column_pull]

use_df = pd.get_dummies(use_df, columns = onehot_columns, drop_first= True)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
use_df['churn_label'] = label_encoder.fit_transform(use_df['churn_label'])
use_df["churn_label"] = use_df["churn_label"].astype(bool)

In [594]:
use_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6991 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   churn_label                   6991 non-null   bool   
 1   total_refunds                 6991 non-null   float64
 2   tenure_months                 6991 non-null   int64  
 3   num_referrals                 6991 non-null   int64  
 4   contract_type_One Year        6991 non-null   bool   
 5   contract_type_Two Year        6991 non-null   bool   
 6   has_dependents_1              6991 non-null   bool   
 7   has_premium_tech_support_Yes  6991 non-null   bool   
dtypes: bool(5), float64(1), int64(2)
memory usage: 252.6 KB


In [623]:
use_df["churn_label"].value_counts()
# full_df["churn_label"].value_counts()

churn_label
False    5174
True     1817
Name: count, dtype: int64

In [596]:

use_df = use_df.rename(
    columns={
        'contract_type_One Year': 'contract_type_One_Year', 'contract_type_Two Year': 'contract_type_Two_Year'})

In [597]:
feature_list_hot = use_df.columns.tolist()

# remove churn_label column
feature_list_hot = feature_list_hot[1:]

print(feature_list_hot)


['total_refunds', 'tenure_months', 'num_referrals', 'contract_type_One_Year', 'contract_type_Two_Year', 'has_dependents_1', 'has_premium_tech_support_Yes']


In [598]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import numpy as np


In [599]:
X = use_df[feature_list_hot]
y = use_df["churn_label"]


kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation loop
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3,random_state=42)

In [600]:
# Experiment 1: Catboost model

model = CatBoostClassifier(learning_rate = 0.1, verbose=0)

model.fit(X_train,y_train)

y_prob = model.predict_proba(X_test)[:, 1]

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print("Accuracy:", accuracy)
print("confusion matrix:", conf_matrix)
print("auc score:", auc)

Accuracy: 0.7982832618025751
confusion matrix: [[903 147]
 [135 213]]
auc score: 0.8547796934865901


In [601]:
# Experiment 2: XGBoost model

xgb_model = XGBClassifier(learning_rate = 0.1, verbosity = 0)

xgb_model.fit(X_train,y_train)

y_prob = xgb_model.predict_proba(X_test)[:, 1]

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print("Accuracy:", accuracy)
print("confusion matrix:", conf_matrix)
print("auc score:", auc)


Accuracy: 0.8125894134477826
confusion matrix: [[951  99]
 [163 185]]
auc score: 0.8693787629994526


In [ ]:
# XGBoost has higher AUC score. Proceed with XGBoost for hyperparameter optimization

In [ ]:
#xgboost

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

# Define the model
model = XGBClassifier(eval_metric='auc')

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 2, 3],
}


# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring="roc_auc",
    n_iter=50,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters and AUC score
print("Best Parameters:", random_search.best_params_)
print("Best AUC Score:", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 2, 'reg_alpha': 0.01, 'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 1.0}
Best AUC Score: 0.8700349029666862


In [ ]:
# Use selected parameter for final model

final_xgb_model = XGBClassifier(subsample = 1.0, reg_lambda = 2,
                                reg_alpha = 0.01, n_estimators = 100,
                                min_child_weight = 5, max_depth = 3,
                                learning_rate = 0.05, gamma = 0.2,
                                colsample_bytree = 1.0, verbosity = 0)

final_xgb_model.fit(X_train,y_train)

y_prob = xgb_model.predict_proba(X_test)[:, 1]

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print("Accuracy:", accuracy)
print("confusion matrix:", conf_matrix)
print("auc score:", auc)


Accuracy: 0.8125894134477826
confusion matrix: [[951  99]
 [163 185]]
auc score: 0.8693787629994526


In [606]:
from pathlib import Path

Path("../model").mkdir(exist_ok=True)  # Create model/ directory if doesn't exist



In [607]:
import joblib

joblib.dump(final_xgb_model, '../model/final_xgb_model.pkl')

['../model/final_xgb_model.pkl']